In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv(r"C:\Users\Sahil\Downloads\archive (7)\songdata.csv")
df.head(3)

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \nAnd..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \nTouch me gentl..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \nWhy I had t...


In [3]:
df.shape

(57650, 4)

In [4]:
df = df.sample(n=5000).drop('link', axis=1).reset_index(drop=True)

In [5]:
df.shape

(5000, 3)

# cleaning

In [18]:
df['text'][0]

"there 's red in my head but i do n't want it the blue were swarm there in my soul shadow over me and the onli colour i can paint my soul i want black and do n't fade away i want it black forev insid of me , all i want to see 's the colour in my head i 'm in black the onli way and make it black forev go insid and go insid and never come back out again do n't colour me 'caus i do n't want it the raven taken over my soul the onli thing i want is the onli way i want to go the red were in my head , the blue were my soul swirl in my mind , never let go blind my eye with a cold embrac i 'll lay my veil all across my face everyon i use to know see me strang , see me bold everyth i use to hold will fade away my soul no confess and no regret i 'll paint the black all across my chest gim me dark and gim me rest gim me black or gim me death"

In [6]:
df['text'] = df['text'].str.lower().replace(r'[^\w\s]','').replace(r'\n',' ', regex=True)

In [19]:
df['text'][0]

"there 's red in my head but i do n't want it the blue were swarm there in my soul shadow over me and the onli colour i can paint my soul i want black and do n't fade away i want it black forev insid of me , all i want to see 's the colour in my head i 'm in black the onli way and make it black forev go insid and go insid and never come back out again do n't colour me 'caus i do n't want it the raven taken over my soul the onli thing i want is the onli way i want to go the red were in my head , the blue were my soul swirl in my mind , never let go blind my eye with a cold embrac i 'll lay my veil all across my face everyon i use to know see me strang , see me bold everyth i use to hold will fade away my soul no confess and no regret i 'll paint the black all across my chest gim me dark and gim me rest gim me black or gim me death"

In [20]:
import nltk
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

def tokenization(txt):
    tokens = nltk.word_tokenize(txt)
    stemming = [stemmer.stem(w) for w in tokens]
    return " ".join(stemming)

In [21]:
df['text'] = df['text'].apply(lambda x: tokenization(x))

In [22]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [23]:
tfidvector = TfidfVectorizer(analyzer='word',stop_words='english')
matrix = tfidvector.fit_transform(df['text'])
similarity = cosine_similarity(matrix)

In [24]:
similarity[0]

array([1.        , 0.02981758, 0.08110849, ..., 0.0533931 , 0.03578409,
       0.01330176])

In [31]:
df[df['song']=='Black Forever']

,artist,song,text
0,W.A.S.P.,Black Forever,there 's red in my head but i do n't want it t...


# recommedation function

In [29]:
def recommendation(song_df):
    idx = df[df['song'] == song_df].index[0]
    distances = sorted(list(enumerate(similarity[idx])),reverse=True,key=lambda x:x[1])
    
    songs = []
    for m_id in distances[1:21]:
        songs.append(df.iloc[m_id[0]].song)
        
    return songs

In [30]:
df['song']

0                      Black Forever
1       By The Fruits Of Their Labor
2                    Talk About Love
3                         The Answer
4                       Delia's Gone
                    ...             
4995              Mushmouth Shoutin'
4996                    Lazy Old Sun
4997                   I Didn't Know
4998              Conversation Peace
4999                     Gypsy Queen
Name: song, Length: 5000, dtype: object

In [32]:
recommendation('Black Forever')

['Swamp Song',
 'Paint It Black',
 'Simple Pages',
 'Modern Girl',
 'Fuel',
 'Country Honk',
 'Black Eye',
 'Black Star',
 'Sunset And Babylon',
 'I Want To Know You (More)',
 'Black And Gold',
 'If Only',
 'Black Metal',
 'The Blues In Black And White',
 'Just A Little Bit',
 'Black Gloves',
 'Black And Yellow',
 "Don't Want To Know",
 'Wanna Be There',
 'Black Rage']

In [34]:
import pickle
pickle.dump(similarity,open('similarity_song.pkl','wb'))
pickle.dump(df,open('df.pkl','wb'))

In [35]:
df.head()

,artist,song,text
0,W.A.S.P.,Black Forever,there 's red in my head but i do n't want it t...
1,Linda Ronstadt,By The Fruits Of Their Labor,take the time to look around you look what 's ...
2,Judds,Talk About Love,"[ choru ] love , talk about love whi do you al..."
3,Incubus,The Answer,"( mm , hemp ? ) step outsid the bound and take..."
4,Johnny Cash,Delia's Gone,"delia , oh . delia delia all my life if i had ..."


In [36]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

In [37]:
# Preprocess the lyrics text
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

In [39]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Sahil\AppData\Roaming\nltk_data...


True

In [41]:
nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Sahil\AppData\Roaming\nltk_data...


True

In [42]:
def preprocess_text(text):
    tokens = word_tokenize(text.lower())
    tokens = [lemmatizer.lemmatize(token) for token in tokens if token.isalpha()]
    tokens = [token for token in tokens if token not in stop_words]
    return ' '.join(tokens)

df['preprocessed_lyrics'] = df['text'].apply(preprocess_text)

In [45]:
from textblob import TextBlob
import pandas as pd

df['Sentiment'] = df['text'].apply(lambda x: TextBlob(x).sentiment.polarity)

df['Sentiment_Classification'] = df['Sentiment'].apply(lambda x: 'Positive' if x > 0 else 'Negative' if x < 0 else 'Neutral')

In [89]:
for index, row in df.iterrows():
    print(f"Song: {row['song']}")
    print(f"Sentiment Polarity: {row['Sentiment']}")
    print(f"Sentiment Classification: {row['Sentiment_Classification']}")
    print("-----------------------------")

Song: Black Forever
Sentiment Polarity: -0.1277777777777778
Sentiment Classification: Negative
-----------------------------
Song: By The Fruits Of Their Labor
Sentiment Polarity: 0.4071428571428571
Sentiment Classification: Positive
-----------------------------
Song: Talk About Love
Sentiment Polarity: 0.3531250000000001
Sentiment Classification: Positive
-----------------------------
Song: The Answer
Sentiment Polarity: 0.10997474747474746
Sentiment Classification: Positive
-----------------------------
Song: Delia's Gone
Sentiment Polarity: -0.004987373737373743
Sentiment Classification: Negative
-----------------------------
Song: Before The Next Teardrop Falls
Sentiment Polarity: 0.25
Sentiment Classification: Positive
-----------------------------
Song: Elephants And Flowers
Sentiment Polarity: 0.20180555555555557
Sentiment Classification: Positive
-----------------------------
Song: King Of The Road
Sentiment Polarity: -0.02375
Sentiment Classification: Negative
---------------

In [50]:
from sklearn.cluster import KMeans
# Extract song texts
song_texts = df['text']

# Initialize the TF-IDF vectorizer
vectorizer = TfidfVectorizer()

# Convert the song texts into a TF-IDF matrix
X = vectorizer.fit_transform(song_texts)

# Perform clustering using K-means
k = 5  # Specify the number of desired clusters
kmeans = KMeans(n_clusters=k, random_state=42)
kmeans.fit(X)

# Get the cluster labels assigned to each song
cluster_labels = kmeans.labels_

# Print the cluster labels for each song
for i in range(len(df)):
    print(f"Song: {df['song'][i]}, Cluster: {cluster_labels[i]}")

Song: Black Forever, Cluster: 2
Song: By The Fruits Of Their Labor, Cluster: 2
Song: Talk About Love, Cluster: 3
Song: The Answer, Cluster: 2
Song: Delia's Gone, Cluster: 4
Song: Before The Next Teardrop Falls, Cluster: 3
Song: Elephants And Flowers, Cluster: 1
Song: King Of The Road, Cluster: 4
Song: The Kiss Of Kathleen Turner, Cluster: 4
Song: I Gotta Know, Cluster: 3
Song: Those Were The Days, Cluster: 2
Song: Love Runs Out, Cluster: 2
Song: Nothing Can Change This Love, Cluster: 3
Song: I Drove Her Out Of My Mind, Cluster: 0
Song: Devil's Island, Cluster: 2
Song: Mundzumundbeatmung, Cluster: 4
Song: Evil, Cluster: 3
Song: Steer Your Way, Cluster: 4
Song: He's Gonna Step On You Again, Cluster: 1
Song: Ball And Chain, Cluster: 2
Song: Give Your Heart A Break, Cluster: 3
Song: There's No Tomorrow, Cluster: 4
Song: Beautiful, Cluster: 2
Song: I Can Read Your Mind, Cluster: 3
Song: Isn't It A Pity, Cluster: 2
Song: Gemini Twin, Cluster: 2
Song: Hatfield, Cluster: 2
Song: Ave Maria, Clu

In [55]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans


# Extract song texts
song_texts = df['text']

# Initialize the TF-IDF vectorizer
vectorizer = TfidfVectorizer()

# Convert the song texts into a TF-IDF matrix
X = vectorizer.fit_transform(song_texts)

# Perform clustering using K-means
k = 5  # Specify the number of desired clusters
kmeans = KMeans(n_clusters=k, random_state=42)
kmeans.fit(X)

# Get the cluster labels assigned to each song
cluster_labels = kmeans.labels_

# Create a new DataFrame with song and cluster information
song_clusters = pd.DataFrame({'Song': df['song'], 'Cluster': cluster_labels})

# Convert the cluster labels to a NumPy array for sorting
song_clusters_np = song_clusters.values

# Sort the array based on the cluster labels
song_clusters_sorted_np = song_clusters_np[np.argsort(song_clusters_np[:, 1])]

# Convert the sorted NumPy array back to a DataFrame
song_clusters_sorted = pd.DataFrame(song_clusters_sorted_np, columns=song_clusters.columns)

# Print the sorted DataFrame
print(song_clusters_sorted)


                                   Song Cluster
0                       Young Birds Fly       0
1                         Harbour Lecou       0
2                        And I Love Her       0
3                          Emerald Eyes       0
4                                   She       0
...                                 ...     ...
4995                  Too Much Too Soon       4
4996                       Antenna Head       4
4997            Force Ten (Phoenix, AZ)       4
4998  Rockin' Around The Christmas Tree       4
4999                        Gypsy Queen       4

[5000 rows x 2 columns]


In [70]:
song_clusters_sorted.head()

,Song,Cluster
0,Young Birds Fly,0
1,Harbour Lecou,0
2,And I Love Her,0
3,Emerald Eyes,0
4,She,0


In [81]:
df = df.merge(song_clusters_sorted[['Song', 'Cluster']], left_on='song', right_on='Cluster', how='left')

In [83]:
df = df.drop(['Song_x', 'Cluster_y','Song_y'], axis=1)

In [88]:
df.head()

,artist,song,text,preprocessed_lyrics,Sentiment,Sentiment_Classification,Cluster_x,genre
0,W.A.S.P.,Black Forever,there 's red in my head but i do n't want it t...,red head want blue swarm soul shadow onli colo...,-0.127778,Negative,2,Psychedelic Rock
1,Linda Ronstadt,By The Fruits Of Their Labor,take the time to look around you look what 's ...,take time look around look grow seed sow take ...,0.407143,Positive,2,Psychedelic Rock
2,Judds,Talk About Love,"[ choru ] love , talk about love whi do you al...",choru love talk love whi alway want talk love ...,0.353125,Positive,3,rock n roll
3,Incubus,The Answer,"( mm , hemp ? ) step outsid the bound and take...",mm hemp step outsid bound take big look time l...,0.109975,Positive,2,Psychedelic Rock
4,Johnny Cash,Delia's Gone,"delia , oh . delia delia all my life if i had ...",delia oh delia delia life shot poor delia wife...,-0.004987,Negative,4,romantic


In [86]:
import pandas as pd
# Map cluster labels to genres
cluster_genre_mapping = {4: 'romantic', 0: 'rock',3:'rock n roll',2:'Psychedelic Rock',1:'rock anthem'}

# Add a new column 'genre' based on cluster labels
df['genre'] = df['Cluster_x'].map(cluster_genre_mapping)

# Print the genres of songs in cluster 4 and 0
cluster_4_songs = df[df['Cluster_x'] == 4]['song']
cluster_0_songs = df[df['Cluster_x'] == 0]['song']
cluster_2_songs = df[df['Cluster_x'] == 2]['song']
cluster_1_songs = df[df['Cluster_x'] == 1]['song']
print("Songs in Cluster 4 (Romantic):")
print(cluster_4_songs)

print("\nSongs in Cluster 0 (Rock):")
print(cluster_0_songs)

print("\nSongs in Cluster 1 (rock anthem):")
print(cluster_1_songs)

print("\nSongs in Cluster 2 (Psychedelic Rock):")
print(cluster_2_songs)


Songs in Cluster 4 (Romantic):
4                      Delia's Gone
7                  King Of The Road
8                  King Of The Road
9                  King Of The Road
10      The Kiss Of Kathleen Turner
                   ...             
5585    Believe In The Supernatural
5586                    Dawn Patrol
5588                   Lazy Old Sun
5589                  I Didn't Know
5591                    Gypsy Queen
Name: song, Length: 1914, dtype: object

Songs in Cluster 0 (Rock):
15              I Drove Her Out Of My Mind
51               Straight Through My Heart
64                  Little Miss Understood
81      I Gave It Up (When I Fell In Love)
95                       Living In The End
                       ...                
5530              Did You Not Hear My Lady
5534                      Drunk On Shadows
5568                       Castles Burning
5581                               Suzanne
5582                               Suzanne
Name: song, Length: 341, dtype: 

In [91]:
import pickle
pickle.dump(similarity,open('similarity_song.pkl','wb'))


In [94]:
pickle.dump(df,open('df.pkl','wb'))